In [1]:
%cd '/content/drive/My Drive/Colab data/ROBERTA'
!pip install omegaconf hydra-core fairseq
!git clone https://github.com/pytorch/fairseq.git
!nvidia-smi

     |████████████████████████████████| 1.7 MB 550 kB/s eta 0:00:01
     |████████████████████████████████| 64 kB 1.3 MB/s eta 0:00:011
     |████████████████████████████████| 122 kB 4.1 MB/s eta 0:00:01     |█████████████▍                  | 51 kB 4.4 MB/s eta 0:00:01
     |████████████████████████████████| 112 kB 4.2 MB/s eta 0:00:01
  Using cached PyYAML-5.3.1.tar.gz (269 kB)
     |████████████████████████████████| 2.0 MB 3.9 MB/s eta 0:00:01
  Using cached dataclasses-0.8-py3-none-any.whl (19 kB)
     |████████████████████████████████| 14.8 MB 2.7 MB/s eta 0:00:01    |████████████████                | 7.4 MB 95 kB/s eta 0:01:18     |████████████████▋               | 7.7 MB 95 kB/s eta 0:01:15     |███████████████████████████▏    | 12.6 MB 108 kB/s eta 0:00:21     |███████████████████████████████▌| 14.6 MB 2.7 MB/s eta 0:00:01
     |████████████████████████████████| 723 kB 3.7 MB/s eta 0:00:01
  Using cached torch-1.7.1-cp36-cp36m-manylinux1_x86_64.whl (776.8 MB)
     |█████████████

In [2]:
!wget -O encoder.json https://dl.fbaipublicfiles.com/fairseq/gpt2_bpe/encoder.json
!wget -O vocab.bpe https://dl.fbaipublicfiles.com/fairseq/gpt2_bpe/vocab.bpe
!wget -O dict.txt https://dl.fbaipublicfiles.com/fairseq/gpt2_bpe/dict.txt

--2021-01-06 13:08:26--  https://dl.fbaipublicfiles.com/fairseq/gpt2_bpe/encoder.json
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 104.22.74.142, 172.67.9.4, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1042301 (1018K) [text/plain]
Saving to: ‘encoder.json’

encoder.json        100%[===================>]   1018K  1000KB/s    in 1.0s    

2021-01-06 13:08:29 (1000 KB/s) - ‘encoder.json’ saved [1042301/1042301]

--2021-01-06 13:08:29--  https://dl.fbaipublicfiles.com/fairseq/gpt2_bpe/vocab.bpe
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.74.142, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 456318 (446K) [text/plain]
Saving to: ‘vocab.bpe’

vocab.bpe           100%[===================>] 445.62K   5

In [ ]:
LINES= !cat ROMAN_URDU_DATASET.txt | wc -l
LINES = int(LINES[0]) + 1
TRAIN=int(498*LINES/500)
TEST=int(LINES/500)

In [ ]:
import os
os.system('{ head -n '+str(TRAIN)+' > ROMAN_URDU_DATASET.train.txt; head -n  '+str(TEST)+' > ROMAN_URDU_DATASET.valid.txt;   head -n  '+str(TEST)+' > ROMAN_URDU_DATASET.test.txt; } < ROMAN_URDU_DATASET.txt')

In [ ]:
%cd 'fairseq'
!for SPLIT in train valid test; do \
  python -m examples.roberta.multiprocessing_bpe_encoder \
        --encoder-json ../encoder.json \
        --vocab-bpe ../vocab.bpe \
        --inputs ../ROMAN_URDU_DATASET.${SPLIT}.txt \
        --outputs ../ROMAN_URDU_DATASET.${SPLIT}.bpe \
        --keep-empty \
        --workers 60; \
done


In [ ]:
!fairseq-preprocess \
    --only-source \
    --srcdict ../dict.txt \
    --trainpref ../ROMAN_URDU_DATASET.train.bpe \
    --validpref ../ROMAN_URDU_DATASET.valid.bpe \
    --testpref ../ROMAN_URDU_DATASET.test.bpe \
    --destdir ../data-bin/ \
    --workers 60

In [ ]:
TOTAL_UPDATES=125000    # Total number of training steps
WARMUP_UPDATES=10000    # Warmup the learning rate over this many updates
PEAK_LR=0.0005          # Peak learning rate, adjust as needed
TOKENS_PER_SAMPLE=512   # Max sequence length
MAX_POSITIONS=512       # Num. positional embeddings (usually same as above)
MAX_SENTENCES=16        # Number of sequences per batch (batch size)
UPDATE_FREQ=16          # Increase the batch size 16x

DATA_DIR='data-bin'

!fairseq-train --fp16 $DATA_DIR \
    --task masked_lm --criterion masked_lm \
    --arch roberta_base --sample-break-mode complete --tokens-per-sample $TOKENS_PER_SAMPLE \
    --optimizer adam --adam-betas '(0.9,0.98)' --adam-eps 1e-6 --clip-norm 0.0 \
    --lr-scheduler polynomial_decay --lr $PEAK_LR --warmup-updates $WARMUP_UPDATES --total-num-update $TOTAL_UPDATES \
    --dropout 0.1 --attention-dropout 0.1 --weight-decay 0.01 \
    --batch-size $MAX_SENTENCES --update-freq $UPDATE_FREQ \
    --max-update $TOTAL_UPDATES --log-format simple --log-interval 1
